## Save and load the model

In [11]:
import tensorflow as tf
from amn.model.aMNWtModel import AMNWtModel, RNNCell
model_class = AMNWtModel


# from aMRNNModel import AMRNNModel, RNNCell
# model_class = AMRNNModel
# model_file = "Models/AMRNN_model.keras"


seed = 10
tf.random.set_seed(seed)

data_dir = "../data"

# dataset_file = "/Dataset/IJN1463_10_UB.npz"
# objective=['BIOMASS_KT2440_WT3']
# epoch = 20
# batch_size = 30
# uptake_max_index = None
# model_file = "Models/AMNWt_IJN1463_10_UB.keras"
# model_file = "Models/AMNWt_IJN1463_10_UB_no_scaling.keras"


dataset_file = "/Dataset/e_coli_core_UB.npz"
objective=['BIOMASS_Ecoli_core_w_GAM']
epoch = 20 #200
batch_size = 7
uptake_max_index = None
model_file = "Models/AMNWt_e_coli_core_UB.keras"
model_file = "Models/AMNWt_e_coli_core_UB_no_scaling.keras"


# dataset_file = "/Dataset/biolog_iML1515_EXP_UB.npz"
# objective=['BIOMASS_Ec_iML1515_core_75p37M']
# epoch = 1 #20
# batch_size = 30
# uptake_max_index=151

# Dataset plus model structure
print("---------------------------------------- model ----------------------------------------")
model = model_class(dataset_file=data_dir + dataset_file, 
                   objective=objective,
                   timestep=4,
                   hidden_dim=50,
                   verbose=True,
                )
model.printout()

# Preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler 
from amn.tools import MaxScaler
scaler= MaxScaler()
model.train_test_split(test_size=0.1, random_state=seed)
model.preprocess(scaler)
model.preprocessing_for_specific_model()


# Construct and train an AMNWt model
AMNWt_model = model.build_model()
history = AMNWt_model.fit(model.X_train, model.Y_train, epochs=epoch, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model.predict(model.X_test)))

---------------------------------------- model ----------------------------------------
number of metabolites:  72
filtered measurements size:  1
dataset file: ../data/Dataset/e_coli_core_UB.npz
model type: AMNWt
model medium bound: UB
timestep: 4
training set size (1000, 20) (1000, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu
R2 : 0.9644429809187083
Q2 : 0.9606012550259331


In [5]:
from amn.tools import custom_loss

seed = 10
tf.random.set_seed(seed)

# Recreate new model from config file, compile and train it. First test on config.
config = AMNWt_model.get_config()
AMNWt_model_= tf.keras.Model.from_config(config, custom_objects={"RNNCell":RNNCell})
my_mse = custom_loss(model.S, model.P_out,model.P_in)
AMNWt_model_.compile(loss=my_mse,optimizer='adam',metrics=[my_mse])
history = AMNWt_model_.fit(model.X_train, model.Y_train, epochs=epoch, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))

R2 : 0.9644429809187083
Q2 : 0.9606012550259331


In [8]:
# Save the train AMNWt_model
model_file = "../models/AMNWt_e_coli_core_UB_to_delete.keras"
tf.keras.models.save_model(AMNWt_model,model_file, overwrite=True, save_format=None, save_traces=True)

In [9]:
from amn.tools import custom_loss

AMNWt_model_ = tf.keras.models.load_model(model_file, 
                                          custom_objects={"RNNCell":RNNCell,
                                                          "my_mse":custom_loss(model.S, 
                                                                               model.P_out,
                                                                               model.P_in)}
                                          )

print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))
history = AMNWt_model_.fit(model.X_train, model.Y_train, epochs=epoch, batch_size=batch_size, verbose=0)
print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))

R2 : 0.9644429809187083
Q2 : 0.9606012550259331
R2 : 0.974495519143497
Q2 : 0.9710487697486336
